First, we set up a function that generates $N$ cities randomly in a $1\times1$ square box and calculates the distance between each two of them. The input is the number of cities $N$, and the output is an $N\times N$ matrix with its $i,j$ element being the distance between city $i$ and $j$.

In [5]:
import numpy as np
import random

def generate_cities_and_distances(N):
    # Generate N cities randomly in a 1x1 square box
    cities = np.random.rand(N, 2)  # Each city is represented by its (x, y) coordinates
    
    # Calculate the distances between each pair of cities
    distances = np.zeros((N, N))  # Initialize an N x N matrix for distances
    for i in range(N):
        for j in range(N):
            distances[i][j] = np.sqrt(np.sum((cities[i] - cities[j]) ** 2))  # Calculate Euclidean distance
            
    return distances

The next function generates an initial guess for the traveling salesman's shortest route. Starting from city $i=0$, find its nearest neighbour $j$; then find $j$'s nearest neighbour except for $i$, let's say it's $k$; then find $k$'s nearest neighbour $l$ except for $i$ and $j$; keep doing this until each of the $N$ cities has appeared once. This function takes the distance matrix as input and gives an $N$-element sequence recording the route as output.

In [ ]:
def generate_initial_route(distances):
    N = len(distances)  # Number of cities
    route = [0]  # Start from city 0
    visited = set([0])  # Keep track of visited cities
    
    while len(route) < N:
        # Find the nearest neighbour of the last city in the route that hasn't been visited yet
        min_dist = float('inf')  # Initialize to a large value
        next_city = -1  # Index of the next city to add to the route
        for j in range(N):
            if j not in visited:  # Ignore already visited cities
                dist = distances[route[-1]][j]  # Distance from the last city in the route to city j
                if dist < min_dist:  # Update the minimum distance and next city
                    min_dist = dist
                    next_city = j
        route.append(next_city)  # Add the next city to the route
        visited.add(next_city)  # Mark the city as visited
    
    return route  # The initial route as a sequence of city indices